In [14]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [13]:
from langchain import hub
from langchain.document_loaders import DirectoryLoader #, MarkdownLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# Path to your Obsidian markdown notes
directory_path = "../../data/lesha_obsi_sample"

# Initialize the DirectoryLoader to load .md files
loader = DirectoryLoader(
    directory_path,
    glob="**/*.md",
    loader_cls=UnstructuredMarkdownLoader,
    loader_kwargs={"encoding": "utf-8"}
)

docs = loader.load()
print(len(docs))
print(docs[0])

110
page_content='Связи: Теги:

Самое дешевое вино. Ибо самая большая микрозона. Алазанская долина - посути микс любых виноградов с долины. Поэтому его запасы не ограничены и оно самое дешевое - материал неограничен.' metadata={'source': '../../data/lesha_obsi_sample/Алазанская долина (wine).md'}


In [10]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Print the splits to verify
print(len(splits))
for split in splits[:5]:
    print(split.page_content[:200])  # Print first 200 characters of each split

147
Связи: Теги:

Самое дешевое вино. Ибо самая большая микрозона. Алазанская долина - посути микс любых виноградов с долины. Поэтому его запасы не ограничены и оно самое дешевое - материал неограничен.
Связи: Теги: tags: - "#Book" - "#SelfDev"

Рассмотреть, как моя рациональная и импульсивная личность ведут себя во время [[СВ_Испытание силы я буду]], [[СВ_Испытание силы я не буду]]

1) Во время испы
tags: links:

Basics
Связи: Теги: #Продуктивность
Связи: [[SuperVised Learning]] Теги: #ML

Задача [[Second brain/main/Supervised learning|Supervised learning]] машинного обучения, заключающаяся в том, чтоб разбить входные данные на 2 класса, бинарно


In [11]:
#### Retriever
# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [15]:
#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")
print(f"prompt: {prompt}")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
rag_chain.invoke("Расскажи о винах Грузии?")


prompt: input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


'Грузия известна своими винами, в том числе белым полусладким вином. В Грузии используют уникальный способ приготовления вина с использованием квеври. На застольях в Грузии пьют один напиток, предпочтительно белое вино.'